In [7]:
import q1k_sync_tools as qst
import mne
mne.viz.set_browser_backend('qt')
%matplotlib qt
import pandas as pd
pd.options.mode.chained_assignment = None
import pylossless as ll
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import scipy

## Group analysis

In [2]:
subjects = ['002','004','006','008','009','010','012','013','015']

In [3]:
peak_itc_alpha = []
gc_events = ['gcoc','gcgc']
for subject in subjects:
    for gc_event in gc_events:
        subject_raw_eeg= glob.glob(f'sourcedata/{subject}_1/{subject}_1_eeg/{subject}_1_go_*.mff')[0]
        subject_derivative = f'derivatives/pylossless/sub-{subject}/eeg/sub-{subject}_task-go_eeg.edf'
        
        eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)
        eeg_events = mne.find_events(eeg_raw, shortest_event=1)
        eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)
        
        #eeg_event_dict['flash_din'] = eeg_event_dict.pop('DIN3')
        
        mask = np.isin(eeg_events[:,2],[eeg_event_dict[gc_event]])
        eeg_events = eeg_events[mask]
        
        ll_state = ll.LosslessPipeline()
        ll_state = ll_state.load_ll_derivative(subject_derivative)
        
        ll_qcr = ll_state.raw.copy()
        manual = []
        for flag_type in ll_state.flags['ch']:
            manual.extend(ll_state.flags['ch'][flag_type])
        print(ll_state.flags['ch'])
        ll_qcr.info['bads'].extend(manual)
        ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))
        
        df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
        ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)
        
        ll_qcr.load_data()
        ll_state.ica2.apply(ll_qcr)
        ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
        ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
        
        
        epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
        freqs = np.array([8,9,10,11,12,13])
        n_cycles = freqs / 2.0  # different number of cycle per frequency
        power, itc = mne.time_frequency.tfr_morlet(
            epochs,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=True,
            return_itc=True,
            decim=3,
            n_jobs=None,
        )
        peak_itc = pd.DataFrame([itc.data.max()])
        peak_itc['gc_event']=gc_event
        peak_itc['subject']=subject
        peak_itc_alpha.append(peak_itc)

Reading EGI MFF Header from /Users/lina_01/acar-q1k-pilot-seg/sourcedata/002_1/002_1_eeg/002_1_go_20230228_031409.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {VBeg} ...
412 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
Extracting EDF parameters from /Users/lina_01/acar-q1k-pilot-seg/derivatives/pylossless/sub-002/eeg/sub-002_task-go_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from derivatives/pylossless/sub-002/eeg/sub-002_task-go_events.tsv.
Reading channel info from derivatives/pylossless/sub-002/eeg/sub-002_task-go_channels.tsv.
Reading electrode coords from derivatives/pylossless/sub-002/eeg/sub-002_space-CapTrak_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading derivatives/pylossless/sub-002/eeg/sub-002_task-go_ica1_ica.fif ...
Now r

/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: The number of channels in the channels.tsv sidecar file (157) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN3, DIN4, STI 014, TSYN, VBeg, dbbc, dbgc, dboc, dfbc, dfgc, dfoc, dggc, drbc, drgc, droc, dsbc, dsgc, dsoc, dtbc, dtgc, dtoc, gcbc, gcgc, gcoc, geoc, gfbc, gfgc, gfoc
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Ready.
{'manual': [], 'bridge': array(['E12', 'E30', 'E35', 'E36', 'E40', 'E50', 'E51', 'E52', 'E57',
       'E62', 'E67', 'E71', 'E72', 'E81', 'E88'], dtype=object), 'ch_sd': array(['E39', 'E48', 'E56', 'E63', 'E82', 'E94', 'E99'], dtype=object), 'rank': array(['E41'], dtype=object)}
Reading 0 ... 428999  =      0.000 ...   428.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (98 components)
    Zeroing out 34 ICA components
    Projecting back using 102 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff fre

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.5s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 12 events and 2001 original time points ...
0 bad epochs dropped


...

/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: The number of channels in the channels.tsv sidecar file (159) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN2, DIN3, DIN4, TRSP, TSYN, VBeg, dbbc, dbgc, dboc, dfbc, dfgc, dfoc, dggc, drbc, drgc, droc, dsbc, dsgc, dsoc, dtbc, dtgc, dtoc, gcbc, gcgc, gcoc, geoc, gfbc, gfgc, gfoc, gpgc
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Ready.
{'manual': [], 'bridge': array(['E18', 'E19', 'E20', 'E23', 'E24', 'E31', 'E55', 'E80', 'E113',
       'E114', 'E118', 'E120', 'E121', 'E124', 'VREF'], dtype=object), 'ch_sd': array(['E14', 'E21', 'E44', 'E48', 'E49', 'E59', 'E88', 'E99', 'E107'],
      dtype=object), 'rank': array(['E71'], dtype=object)}
Reading 0 ... 384999  =      0.000 ...   384.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (86 components)
    Zeroing out 6 ICA components
    Projecting back using 100 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth:

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 12 events and 2001 original time points ...
0 bad epochs dropped


/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/2558733243.py:37: RuntimeWarning: No matching events found for VBeg (event id 1)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/2558733243.py:37: RuntimeWarning: No matching events found for DIN2 (event id 2)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/2558733243.py:37: RuntimeWarning: No matching events found for DIN4 (event id 3)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/2558733243.py:37: RuntimeWarning: No matching events found for DIN3 (event id 4)
  epochs = mne.Epochs(ll_qcr, e

In [4]:
peak_itc_alpha = pd.concat(peak_itc_alpha)

In [5]:
peak_itc_alpha_gcoc = peak_itc_alpha.loc[peak_itc_alpha.gc_event=='gcoc']
peak_itc_alpha_gcgc = peak_itc_alpha.loc[peak_itc_alpha.gc_event=='gcgc']
peak_itc_alpha_gcgc = peak_itc_alpha_gcgc.rename(columns={0:'peak_itc'})
peak_itc_alpha_gcoc = peak_itc_alpha_gcoc.rename(columns={0:'peak_itc'})

In [8]:
scipy.stats.wilcoxon(np.array(peak_itc_alpha_gcgc['peak_itc']),np.array(peak_itc_alpha_gcoc['peak_itc']))

WilcoxonResult(statistic=6.0, pvalue=0.0546875)

In [9]:
np.mean(peak_itc_alpha_gcgc['peak_itc'])

0.8627710303030409

In [10]:
np.mean(peak_itc_alpha_gcoc['peak_itc'])

0.9069239790517215

In [12]:
np.std(peak_itc_alpha_gcgc['peak_itc'])

0.060068614854977546

In [13]:
np.std(peak_itc_alpha_gcoc['peak_itc'])

0.047107967746997347

In [15]:
mean_power_alpha = []
gc_events = ['gcoc','gcgc']
for subject in subjects:
    for gc_event in gc_events:
        subject_raw_eeg= glob.glob(f'sourcedata/{subject}_1/{subject}_1_eeg/{subject}_1_go_*.mff')[0]
        subject_derivative = f'derivatives/pylossless/sub-{subject}/eeg/sub-{subject}_task-go_eeg.edf'
        
        eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)
        eeg_events = mne.find_events(eeg_raw, shortest_event=1)
        eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)
        
        #eeg_event_dict['flash_din'] = eeg_event_dict.pop('DIN3')
        
        mask = np.isin(eeg_events[:,2],[eeg_event_dict[gc_event]])
        eeg_events = eeg_events[mask]
        
        ll_state = ll.LosslessPipeline()
        ll_state = ll_state.load_ll_derivative(subject_derivative)
        
        ll_qcr = ll_state.raw.copy()
        manual = []
        for flag_type in ll_state.flags['ch']:
            manual.extend(ll_state.flags['ch'][flag_type])
        print(ll_state.flags['ch'])
        ll_qcr.info['bads'].extend(manual)
        ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))
        
        df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
        ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)
        
        ll_qcr.load_data()
        ll_state.ica2.apply(ll_qcr)
        ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
        ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
        
        
        epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
        freqs = np.array([8,9,10,11,12,13])
        n_cycles = freqs / 2.0  # different number of cycle per frequency
        power, itc = mne.time_frequency.tfr_morlet(
            epochs,
            freqs=freqs,
            n_cycles=n_cycles,
            use_fft=True,
            return_itc=True,
            decim=3,
            n_jobs=None,
        )
        mean_power = pd.DataFrame([power.data.mean()])
        mean_power['gc_event']=gc_event
        mean_power['subject']=subject
        mean_power_alpha.append(mean_power)

Reading EGI MFF Header from /Users/lina_01/acar-q1k-pilot-seg/sourcedata/002_1/002_1_eeg/002_1_go_20230228_031409.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {VBeg} ...
412 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
Extracting EDF parameters from /Users/lina_01/acar-q1k-pilot-seg/derivatives/pylossless/sub-002/eeg/sub-002_task-go_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from derivatives/pylossless/sub-002/eeg/sub-002_task-go_events.tsv.
Reading channel info from derivatives/pylossless/sub-002/eeg/sub-002_task-go_channels.tsv.
Reading electrode coords from derivatives/pylossless/sub-002/eeg/sub-002_space-CapTrak_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading derivatives/pylossless/sub-002/eeg/sub-002_task-go_ica1_ica.fif ...
Now r

/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: The number of channels in the channels.tsv sidecar file (157) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN3, DIN4, STI 014, TSYN, VBeg, dbbc, dbgc, dboc, dfbc, dfgc, dfoc, dggc, drbc, drgc, droc, dsbc, dsgc, dsoc, dtbc, dtgc, dtoc, gcbc, gcgc, gcoc, geoc, gfbc, gfgc, gfoc
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Now restoring ICA solution ...
Ready.
{'manual': [], 'bridge': array(['E12', 'E30', 'E35', 'E36', 'E40', 'E50', 'E51', 'E52', 'E57',
       'E62', 'E67', 'E71', 'E72', 'E81', 'E88'], dtype=object), 'ch_sd': array(['E39', 'E48', 'E56', 'E63', 'E82', 'E94', 'E99'], dtype=object), 'rank': array(['E41'], dtype=object)}
Reading 0 ... 428999  =      0.000 ...   428.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (98 components)
    Zeroing out 34 ICA components
    Projecting back using 102 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 12 events and 2001 original time points ...
0 bad epochs dropped


...

/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: The number of channels in the channels.tsv sidecar file (159) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN2, DIN3, DIN4, TRSP, TSYN, VBeg, dbbc, dbgc, dboc, dfbc, dfgc, dfoc, dggc, drbc, drgc, droc, dsbc, dsgc, dsoc, dtbc, dtgc, dtoc, gcbc, gcgc, gcoc, geoc, gfbc, gfgc, gfoc, gpgc
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Ready.
{'manual': [], 'bridge': array(['E18', 'E19', 'E20', 'E23', 'E24', 'E31', 'E55', 'E80', 'E113',
       'E114', 'E118', 'E120', 'E121', 'E124', 'VREF'], dtype=object), 'ch_sd': array(['E14', 'E21', 'E44', 'E48', 'E49', 'E59', 'E88', 'E99', 'E107'],
      dtype=object), 'rank': array(['E71'], dtype=object)}
Reading 0 ... 384999  =      0.000 ...   384.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (86 components)
    Zeroing out 6 ICA components
    Projecting back using 100 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth:

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 12 events and 2001 original time points ...
0 bad epochs dropped


/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/1664329480.py:37: RuntimeWarning: No matching events found for VBeg (event id 1)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/1664329480.py:37: RuntimeWarning: No matching events found for DIN2 (event id 2)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/1664329480.py:37: RuntimeWarning: No matching events found for DIN4 (event id 3)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/1664329480.py:37: RuntimeWarning: No matching events found for DIN3 (event id 4)
  epochs = mne.Epochs(ll_qcr, e

In [18]:
mean_power_alpha = pd.concat(mean_power_alpha)

In [19]:
mean_power_alpha_gcoc = mean_power_alpha.loc[mean_power_alpha.gc_event=='gcoc']
mean_power_alpha_gcgc = mean_power_alpha.loc[mean_power_alpha.gc_event=='gcgc']
mean_power_alpha_gcoc = mean_power_alpha_gcoc.rename(columns={0:'mean_alpha'})
mean_power_alpha_gcgc = mean_power_alpha_gcgc.rename(columns={0:'mean_alpha'})

In [20]:
scipy.stats.wilcoxon(np.array(mean_power_alpha_gcoc['mean_alpha']),np.array(mean_power_alpha_gcgc['mean_alpha']))

WilcoxonResult(statistic=7.0, pvalue=0.07421875)

In [21]:
np.mean(mean_power_alpha_gcoc['mean_alpha'])

1.3278832820616534e-08

In [22]:
np.mean(mean_power_alpha_gcgc['mean_alpha'])

3.320204522635518e-06

In [23]:
np.std(mean_power_alpha_gcgc['mean_alpha'])

6.260235521460853e-06

In [24]:
np.std(mean_power_alpha_gcgc['mean_alpha'])

6.260235521460853e-06

## Single subject graphs

In [25]:
subject_raw_eeg= glob.glob(f'sourcedata/002_1/002_1_eeg/002_1_go_*.mff')[0]
subject_derivative = f'derivatives/pylossless/sub-002/eeg/sub-002_task-go_eeg.edf'

eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)
eeg_events = mne.find_events(eeg_raw, shortest_event=1)
eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)

#eeg_event_dict['flash_din'] = eeg_event_dict.pop('DIN3')

mask = np.isin(eeg_events[:,2],[eeg_event_dict['gcoc']])
eeg_events = eeg_events[mask]

ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(subject_derivative)

ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")


epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')

freqs = np.array([4,5,6,7])
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

Reading EGI MFF Header from /Users/lina_01/acar-q1k-pilot-seg/sourcedata/002_1/002_1_eeg/002_1_go_20230228_031409.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {VBeg} ...
412 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26]
Extracting EDF parameters from /Users/lina_01/acar-q1k-pilot-seg/derivatives/pylossless/sub-002/eeg/sub-002_task-go_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from derivatives/pylossless/sub-002/eeg/sub-002_task-go_events.tsv.
Reading channel info from derivatives/pylossless/sub-002/eeg/sub-002_task-go_channels.tsv.
Reading electrode coords from derivatives/pylossless/sub-002/eeg/sub-002_space-CapTrak_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading derivatives/pylossless/sub-002/eeg/sub-002_task-go_ica1_ica.fif ...
Now r

/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: The number of channels in the channels.tsv sidecar file (157) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/Users/lina_01/acar-q1k-pilot-seg/seg_env/lib/python3.11/site-packages/pylossless/pipeline.py:1342: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN3, DIN4, STI 014, TSYN, VBeg, dbbc, dbgc, dboc, dfbc, dfgc, dfoc, dggc, drbc, drgc, droc, dsbc, dsgc, dsoc, dtbc, dtgc, dtoc, gcbc, gcgc, gcoc, geoc, gfbc, gfgc, gfoc
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Ready.
{'manual': [], 'bridge': array(['E12', 'E30', 'E35', 'E36', 'E40', 'E50', 'E51', 'E52', 'E57',
       'E62', 'E67', 'E71', 'E72', 'E81', 'E88'], dtype=object), 'ch_sd': array(['E39', 'E48', 'E56', 'E63', 'E82', 'E94', 'E99'], dtype=object), 'rank': array(['E41'], dtype=object)}
Reading 0 ... 428999  =      0.000 ...   428.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (98 components)
    Zeroing out 34 ICA components
    Projecting back using 102 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff fre

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.6s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 12 events and 2001 original time points ...
0 bad epochs dropped


/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/640231630.py:33: RuntimeWarning: No matching events found for VBeg (event id 1)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/640231630.py:33: RuntimeWarning: No matching events found for DIN4 (event id 2)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/640231630.py:33: RuntimeWarning: No matching events found for DIN3 (event id 3)
  epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=1.0, on_missing='warn', event_repeated='merge')
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_796/640231630.py:33: RuntimeWarning: No matching events found for TSYN (event id 4)
  epochs = mne.Epochs(ll_qcr, eeg_e

In [26]:
freqs = np.array([8,9,10,11,12,13])
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

Using data from preloaded Raw for 12 events and 2001 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.7s


In [61]:
power.plot(picks='E11',baseline=(-0.5, 0), mode="logratio", title="Average power")

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Applying baseline correction (mode: logratio)


[<Figure size 1280x960 with 2 Axes>]

In [69]:
itc.plot(picks='E11')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
No baseline correction applied


[<Figure size 1280x960 with 2 Axes>]